In [39]:
"""This application demonstrates how to allow a project member to use a
Template in order to create Tags with the Cloud Data Catalog API.

For more information, see the README.md under /datacatalog and the
documentation at https://cloud.google.com/data-catalog/docs.
"""

from googleapiclient.discovery import build
from googleapiclient.http import BatchHttpRequest
from oauth2client.service_account import ServiceAccountCredentials
import uuid

#-------------------------------------------------------------#
# 0. Helper and initialization logic
#-------------------------------------------------------------#

# SET Your environment config.
service_key_file_location = '[SA_PATH]'

project_id = '[MY_PROJECT_ID]'

# Helper container to store results.
class DataContainer:
    def __init__(self):
        self.data = {}

    def callback(self, request_id, response, exception):
        if exception is not None:
            print('request_id: {}, exception: {}'.format(request_id, str(exception)))
            pass
        else:
            print(request_id)
            self.data[request_id] = response


# Helper function to build the the Discovery Service config.         
def get_service(api_name, api_version, scopes, key_file_location):
    """
    Get a service that communicates to a Google API.

    Args:
        api_name: The name of the api to connect to.
        api_version: The api version to connect to.
        scopes: A list auth scopes to authorize for the application.
        key_file_location: The path to a valid service account JSON key file.

    Returns:
        A service that is connected to the specified API.
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service

# Helper function to create a UUID for each request
def generated_uui():
    return str(uuid.uuid4())

def create_batch_request(callback):
    # For more info on supported regions
    # check: https://cloud.google.com/data-catalog/docs/concepts/regions
    
    region='us-datacatalog.googleapis.com'
    
    return BatchHttpRequest(batch_uri='https://{}/batch'.format(region), callback=callback)

container = DataContainer()

# Scope to set up Discovery Service config.
scope = 'https://www.googleapis.com/auth/cloud-platform'

# Create service.
service = get_service(
    api_name='datacatalog',
    api_version='v1',
    scopes=[scope],
    key_file_location=service_key_file_location)


# Create batch request config.
batch = create_batch_request(container.callback)

#-------------------------------------------------------------#
# 1. Start by fetching a list of entries using search call
#-------------------------------------------------------------#

# Create search request body. Example searching for BigQuery tables.
search_request_body = {
  'query': 'type=TABLE system=BIGQUERY',
  'scope': {'includeProjectIds': [project_id]}
}

# Generated a unique id for request.
request_id = generated_uui()

# ADD request to batch client.
batch.add(service.catalog().search(body=search_request_body), request_id=request_id)

# Execute batch request.
batch.execute()

# Uncomment to verify full response from search.
# print(container.data)

response = container.data[request_id]

results = response['results']

first_table = results[0]

# Verify that we have a first table.
print(first_table)

second_table = results[1]

# Verify that we have a second table.
print(second_table)

#-------------------------------------------------------------------#
# 2. Send the attach tags request in bulk for the whole result set
#-------------------------------------------------------------------#

# Create new container
container = DataContainer()

# Create new batch request
batch = create_batch_request(container.callback)

# SET Your template name CONFIG
template_name = 'projects/[MY_PROJECT_ID]/locations/[MY-LOCATION]/tagTemplates/[MY-TEMPLATE-NAME]'

for result in results:
    # Generated a unique id for request.
    request_id = generated_uui()

    # ADD entry name as the Tag parent.
    parent=result['relativeResourceName']

    # Create Tag request body.
    create_tag_request_body = {
      'template': template_name,
       # CHANGE for your Template Field values.
      'fields': {'etl_score': {'doubleValue': 0.5}}
    }

    # ADD request to batch client.
    batch.add(service.projects().locations().
              entryGroups().entries().tags().
              create(body=create_tag_request_body,
                     parent=parent),
              request_id=request_id)

# Execute batch request.

# Since the Batch Client works with regions
# If you receive [HttpError 400 errors]
# 1- verify the region you used to create the Batch client
# 2- verify the region where the Entry is located.
# 3- verify the region of the parent Template used by the Tag.
batch.execute()

# Uncomment to verify full response from create Tag.
# print(container)



FileNotFoundError: [Errno 2] No such file or directory: '[SA_PATH]'